1. Configuração Inicial e LLM

In [ ]:
# Importa bibliotecas necessárias para variáveis de ambiente e o modelo Gemini
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
#.env → arquivo para guardar chaves/senhas/configurações.
# Carrega variáveis de ambiente do arquivo .env
load_dotenv()


In [ ]:
# Obtém a chave da API do Google das variáveis de ambiente
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
# Inicializa o modelo de linguagem grande (LLM) do Google Gemini
# model: Define o modelo Gemini a ser utilizado (ex: "gemini-2.5-flash")
# temperature: Controla a aleatoriedade da resposta (0.0 para respostas mais previsíveis)
# api_key: Chave de autenticação para a API do Google
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
# Exemplo de invocação do LLM para teste
resp_test = llm.invoke("Quem é você? Seja criativo.")

In [ ]:
# # Imprime o conteúdo da resposta do LLM; 
# .content → mostra apenas conteudo
print(resp_test.content)

2. Prompt de Triagem e Saída Estruturada

In [ ]:
# Define o prompt do sistema para o triador de Service Desk
# O prompt instrui o modelo a retornar um JSON com decisão, urgência e campos faltantes
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [ ]:
# Importa classes do Pydantic para definir um modelo de dados estruturado
from pydantic import BaseModel, Field
from typing import Literal, List

In [ ]:
# Define a classe de saída esperada da IA, garantindo a estrutura do JSON
# BaseModel: Classe base para modelos de dados Pydantic
# Field: Permite definir valores padrão ou regras para os campos
# Literal: Garante que o campo aceite apenas valores específicos
# List: Indica que um campo é uma lista
class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)


In [ ]:
#Criando uma nova instância do LLM  para a função triagem

llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
#Importa classes para diferenciar mensagens do sistema e do usuário
#SystemMessage → instruções do sistema (prompt do sistema).
#HumanMessage → mensagens enviadas pelo usuário.

from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
#Dict = dicionário (chave: valor)
#List = lista (coleção de itens)
#Literal = literal (valor fixo, permitido apenas alguns)

from typing import Dict, List, Literal


In [ ]:
# Cria um 'chain' estruturado que força o LLM a produzir saídas no formato TriagemOut
triagem_chain = llm_triagem.with_structured_output(TriagemOut)


In [ ]:
# Define a função de triagem que processa a mensagem do usuário
# Envia a mensagem e o prompt do sistema para o LLM e valida a resposta com TriagemOut

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()


In [ ]:
# Testes da função de triagem
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

3. Processamento de Documentos (RAG)

In [ ]:
# Importa bibliotecas para manipulação de caminhos de arquivo e carregamento de PDFs
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
#cria uma lista vazia que irá guardar todos os documentos extraídos do PDF
docs = []


In [ ]:
# Percorre todos os arquivos PDF na pasta './pdfs/'
# Carrega o texto de cada PDF usando PyMuPDFLoader e adiciona à lista 'd
for n in Path("./pdfs/").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

In [ ]:
# Importa o separador de texto recursivo
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Inicializa o separador de texto para dividir documentos em 'chunks'
# chunk_size: Tamanho máximo de cada pedaço de texto (ex: 300 caracteres)
# chunk_overlap: Quantidade de caracteres que os pedaços se sobrepõem (ex 30 caracteres)

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = splitter.split_documents(docs)


In [ ]:
#Percorre todos os pedaços de texto dos PDFs e imprime cada um, separado por linhas para facilitar a leitura.
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

In [ ]:
#page_content é o texto “de verdade” que você quer que seu agente de IA leia e processe
for chunk in chunks:
    print(chunk.page_content)
    print("------------------------------------")

In [ ]:
# Importa a classe para gerar embeddings do Google Gemini
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
# Inicializa o gerador de embeddings com o modelo Gemini
# Embeddings: Representações numéricas de texto que permitem comparar e buscar textos semanticamente
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [ ]:
# Importa a biblioteca FAISS para armazenamento e busca rápida de vetores
from langchain_community.vectorstores import FAISS
# Cria um Vectorstore FAISS a partir dos chunks e embeddings
vectorstore = FAISS.from_documents(chunks, embeddings)
# Cria um retriever para buscar documentos relevantes com base na similaridade semântica
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [ ]:
# Importa classes para criar prompts de chat e combinar documentos
from langchain_core.prompts import ChatPromptTemplate #permite criar prompts personalizados para chats
from langchain.chains.combine_documents import create_stuff_documents_chain #combina vários chunks/documentos em uma resposta única usando um LLM.


In [ ]:
# Define o prompt para o RAG (Retrieval-Augmented Generation)
# System: Instruções para o modelo (assistente de políticas, responder apenas com contexto)
# Human: Pergunta do usuário e contexto recuperado
prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

In [ ]:
# Cria um 'document_chain' que conecta o LLM ao prompt RAG para processar documentos
document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [ ]:
# Importa bibliotecas para expressões regulares e manipulação de caminhos
import re, pathlib

In [ ]:
def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()
# Funções para formatar as citações dos documentos relevantes

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
# Função principal para perguntar à política usando RAG
# Implementa lógica de segurança: só responde se houver contexto válido
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [ ]:
# Testes da função RAG
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

In [ ]:
# Importa TypedDict e Optional para definir o estado do agente
from typing import TypedDict, Optional
# Define a estrutura do estado do agente usando TypedDict
# Armazena a pergunta, resultados da triagem, resposta, citações, sucesso do RAG e ação final
class AgentState(TypedDict, total = False):
    pergunta: str
    triagem: dict
    resposta: Optional[str]
    citacoes: List[dict]
    rag_sucesso: bool
    acao_final: str


In [ ]:
# Nó de triagem: executa a função de triagem e atualiza o estado
def node_triagem(state: AgentState) -> AgentState:
    print("Executando nó de triagem...")
    return {"triagem": triagem(state["pergunta"])}

In [ ]:
# Nó de auto-resolver: tenta resolver a pergunta usando RAG e atualiza o estado
def node_auto_resolver(state: AgentState) -> AgentState:
    print("Executando nó de auto_resolver...")
    resposta_rag = perguntar_politica_RAG(state["pergunta"])

    update: AgentState = {
        "resposta": resposta_rag["answer"],
        "citacoes": resposta_rag.get("citacoes", []),
        "rag_sucesso": resposta_rag["contexto_encontrado"],
    }

    if resposta_rag["contexto_encontrado"]:
        update["acao_final"] = "AUTO_RESOLVER"

    return update

In [ ]:
# Nó de pedir informação: solicita mais detalhes ao usuário e atualiza o estado

def node_pedir_info(state: AgentState) -> AgentState:
    print("Executando nó de pedir_info...")
    faltantes = state["triagem"].get("campos_faltantes", [])
    if faltantes:
        detalhe = ",".join(faltantes)
    else:
        detalhe = "Tema e contexto específico"

    return {
        "resposta": f"Para avançar, preciso que detalhe: {detalhe}",
        "citacoes": [],
        "acao_final": "PEDIR_INFO"
    }

In [ ]:
# Nó de abrir chamado: simula a abertura de um chamado e atualiza o estado
def node_abrir_chamado(state: AgentState) -> AgentState:
    print("Executando nó de abrir_chamado...")
    triagem = state["triagem"]

    return {
        "resposta": f"Abrindo chamado com urgência {triagem['urgencia']}. Descrição: {state['pergunta'][:140]}",
        "citacoes": [],
        "acao_final": "ABRIR_CHAMADO"
    }

In [ ]:
# Palavras-chave para decidir se um chamado deve ser aberto
KEYWORDS_ABRIR_TICKET = ["aprovação", "exceção", "liberação", "abrir ticket", "abrir chamado", "acesso especial"]

# Função de roteamento: decide o próximo nó após a triagem
def decidir_pos_triagem(state: AgentState) -> str:
    print("Decidindo após a triagem...")
    decisao = state["triagem"]["decisao"]

    if decisao == "AUTO_RESOLVER": return "auto"
    if decisao == "PEDIR_INFO": return "info"
    if decisao == "ABRIR_CHAMADO": return "chamado"

In [ ]:
# Função de roteamento: decide o próximo nó após a tentativa de autoresolução
def decidir_pos_auto_resolver(state: AgentState) -> str:
    print("Decidindo após o auto_resolver...")

    if state.get("rag_sucesso"):
        print("Rag com sucesso, finalizando o fluxo.")
        return "ok"

    state_da_pergunta = (state["pergunta"] or "").lower()

    if any(k in state_da_pergunta for k in KEYWORDS_ABRIR_TICKET):
        print("Rag falhou, mas foram encontradas keywords de abertura de ticket. Abrindo...")
        return "chamado"

    print("Rag falhou, sem keywords, vou pedir mais informações...")
    return "info"

In [ ]:
# Importa classes para construir o grafo de estados
from langgraph.graph import StateGraph, START, END
# Inicializa o workflow como um grafo de estados
workflow = StateGraph(AgentState)
# Adiciona os nós ao grafo
workflow.add_node("triagem", node_triagem)
workflow.add_node("auto_resolver", node_auto_resolver)
workflow.add_node("pedir_info", node_pedir_info)
workflow.add_node("abrir_chamado", node_abrir_chamado)
# Define as arestas (transições) do grafo
workflow.add_edge(START, "triagem")
workflow.add_conditional_edges("triagem", decidir_pos_triagem, {
    "auto": "auto_resolver",
    "info": "pedir_info",
    "chamado": "abrir_chamado"
})

workflow.add_conditional_edges("auto_resolver", decidir_pos_auto_resolver, {
    "info": "pedir_info",
    "chamado": "abrir_chamado",
    "ok": END
})

workflow.add_edge("pedir_info", END)
workflow.add_edge("abrir_chamado", END)
# Compila o grafo para execução
grafo = workflow.compile()

In [ ]:
# Importa bibliotecas para exibir o grafo
from IPython.display import display, Image
# Gera e exibe a imagem do grafo (requer instalação de 'graphviz')
graph_bytes = grafo.get_graph().draw_mermaid_png()
display(Image(graph_bytes))

In [ ]:
# Testes do fluxo completo do agente

testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "É possível reembolsar certificações do Google Cloud?",
          "Posso obter o Google Gemini de graça?",
          "Qual é a palavra-chave da aula de hoje?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
import time

for msg_test in testes:
    resposta_final = grafo.invoke({"pergunta": msg_test})

    triag = resposta_final.get("triagem", {})
    print(f"PERGUNTA: {msg_test}")
    print(f"DECISÃO: {triag.get('decisao')} | URGÊNCIA: {triag.get('urgencia')} | AÇÃO FINAL: {resposta_final.get('acao_final')}")
    print(f"RESPOSTA: {resposta_final.get('resposta')}")
    if resposta_final.get("citacoes"):
        print("CITAÇÕES:")
        for citacao in resposta_final.get("citacoes"):
            print(f" - Documento: {citacao['documento']}, Página: {citacao['pagina']}")
            print(f"   Trecho: {citacao['trecho']}")
    print("------------------------------------")
    # Controla a taxa de requisições para evitar limites da API
 
    time.sleep(8)  # ajuste se ainda der 429
